In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
# for CI testing
smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [45]:
class RegressionModel(nn.Module):
    def __init__(self, indim, outdim):
        # p = number of features
        super(RegressionModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 2, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(2, 1, kernel_size=3, stride=1, padding=1)
        self.linear = nn.Linear(indim, outdim)

    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.linear(y)
#         print('fwd:', y.size())
        return y

regression_model = RegressionModel(10, 5)

In [38]:
print(regression_model.conv1.weight.size(), regression_model.conv1.bias.size())

torch.Size([2, 1, 3]) torch.Size([2])


In [46]:
N = 500  # size of toy data
indim = 10
outdim = 5

def build_linear_dataset(N, indim, outdim, noise_std=0.01):
    X = np.random.rand(N, indim)
    # w = 3
    w = 3 * np.ones((indim, outdim))
    # b = 1
    b = 1*np.ones((N, outdim))
    y = np.matmul(X, w)
#     print(y.shape)
    y = y + b + np.random.normal(0, noise_std, size=(N, outdim))
#     print(y.shape)
    y = y.reshape(N, outdim)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    return data

In [ ]:
build_linear_dataset(N, indim, outdim).size()

In [68]:
def model(data):
    # Create unit normal priors over the parameters
    loc, scale = torch.zeros(outdim, indim), 10 * torch.ones(outdim, indim)
    bias_loc, bias_scale = torch.zeros(outdim), 10 * torch.ones(outdim)
    w_prior = Normal(loc, scale).independent(2)
    print('w_prior shapes: ', w_prior.batch_shape, w_prior.event_shape)
    b_prior = Normal(bias_loc, bias_scale).independent(1)
    print('b_prior shapes: ', b_prior.batch_shape, b_prior.event_shape)
    
    ##for conv layer
    loc1, scale1 = torch.zeros(2, 1, 3), 10*torch.ones(2, 1, 3)
    bias_loc1, bias_scale1 = torch.zeros(2), 10*torch.ones(2)
    w_prior1 = Normal(loc1, scale1).independent(3)
    print('w_prior1 shapes: ', w_prior1.batch_shape, w_prior1.event_shape)
    b_prior1 = Normal(bias_loc1, bias_scale1).independent(1)
    print('b_prior1 shapes: ', b_prior1.batch_shape, b_prior1.event_shape)
    
    #how does sampled prior look?
#     print('how does sampled prior look?')
#     print('w_prior: ', w_prior.sample().size())
#     print('b_prior: ', b_prior.sample().size())
    
    priors = {'linear.weight': w_prior, 'linear.bias': b_prior,
              'conv1.weight': w_prior1, 'conv1.bias': b_prior1}
    # lift module parameters to random variables sampled from the priors
    lifted_module = pyro.random_module("module", regression_model, priors)
    # sample a regressor (which also samples w and b)
    lifted_reg_model = lifted_module()
    with pyro.iarange("map", N):
        x_data = data[:, :-5].view(N,1,-1)
        y_data = data[:, -5:].view(N, 1, -1)
        # run the regressor forward conditioned on data
        prediction_mean = lifted_reg_model(x_data)
#         print('pred_mean shape: ', prediction_mean.size())
#         print('sample shape : ',Normal(prediction_mean, 0.1*torch.ones(data.size(0), 1, outdim)).independent(2).sample().shape)
        # condition on the observed data
        pyro.sample("obs",
                    Normal(prediction_mean, 0.1*torch.ones(data.size(0), 1, outdim)).independent(2),
                    obs=y_data)

In [69]:
softplus = torch.nn.Softplus()

def guide(data):
    # define our variational parameters
    w_loc = torch.randn(outdim, indim)
    # note that we initialize our scales to be pretty narrow
    w_log_sig = torch.tensor(-3.0 * torch.ones(outdim, indim) + 0.05 * torch.randn(outdim, indim))
    b_loc = torch.randn(outdim)
    b_log_sig = torch.tensor(-3.0 * torch.ones(outdim) + 0.05 * torch.randn(outdim))
    ##for conv layer
    w_loc1 = torch.randn(2,1,3)
    w_log_sig1 = torch.tensor(-3.0*torch.ones(2,1,3) + 0.05*torch.randn(2,1,3))
    b_loc1 = torch.randn(2)
    b_log_sig1 = torch.tensor(-3.0*torch.ones(2) + 0.05*torch.randn(2))
  
    # register learnable params in the param store
    mw_param = pyro.param("guide_mean_weight", w_loc)
    sw_param = softplus(pyro.param("guide_log_scale_weight", w_log_sig))
    mb_param = pyro.param("guide_mean_bias", b_loc)
    sb_param = softplus(pyro.param("guide_log_scale_bias", b_log_sig))
    
    mw_param1 = pyro.param("guide_mean_weight1", w_loc1)
    sw_param1 = softplus(pyro.param("guide_log_scale_weight1", w_log_sig1))
    mb_param1 = pyro.param("guide_mean_bias1", b_loc1)
    sb_param1 = softplus(pyro.param("guide_log_scale_bias1", b_log_sig1))
    # guide distributions for w and b
    w_dist = Normal(mw_param, sw_param).independent(2)
    b_dist = Normal(mb_param, sb_param).independent(1)
    w_dist1 = Normal(mw_param1, sw_param1).independent(3)
    b_dist1 = Normal(mb_param1, sb_param1).independent(1)
    dists = {'linear.weight': w_dist, 'linear.bias': b_dist,
            'conv1.weight':w_dist1, 'conv1.bias':b_dist1}
    # overload the parameters in the module with random samples
    # from the guide distributions
    lifted_module = pyro.random_module("module", regression_model, dists)
    # sample a regressor (which also samples w and b)
    return lifted_module()

optim = Adam({"lr": 0.05})
svi = SVI(model, guide, optim, loss=Trace_ELBO())

In [70]:
num_iterations = 10000 if not smoke_test else 2
def main():
    pyro.clear_param_store()
    data = build_linear_dataset(N, 10, 5)
    for j in range(num_iterations):
#         print(j)
        # calculate the loss and take a gradient step
        loss = svi.step(data)
        if j % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (j + 1, loss / float(N)))

if __name__ == '__main__':
    main()

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
[iteration 0001] loss: 58242.9682
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1,

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size(

b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) to

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size(

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size(

...

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
[iteration 9801] loss: -1.4653
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3]

b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) 

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size(

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])
w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size(

w_prior shapes:  torch.Size([]) torch.Size([5, 10])
b_prior shapes:  torch.Size([]) torch.Size([5])
w_prior1 shapes:  torch.Size([]) torch.Size([2, 1, 3])
b_prior1 shapes:  torch.Size([]) torch.Size([2])


In [43]:
for name in pyro.get_param_store().get_all_param_names():
    print((name, pyro.param(name).data.numpy()))

('guide_mean_weight', array([[-1.3590235 , -0.49080193, -1.6997037 , -0.0845492 , -1.0827681 ,
        -1.1413256 , -1.1402293 , -0.15420604, -1.0428376 , -1.8668513 ],
       [-1.2527158 , -0.9333517 , -1.1907883 , -0.2950428 , -1.046036  ,
        -1.4971383 , -0.5154613 , -0.4066267 , -1.4284555 , -1.4575232 ],
       [-1.4263979 , -0.40138605, -1.5928124 , -0.46236512, -0.674542  ,
        -1.4010503 , -0.83601326, -0.7649287 , -0.32026544, -2.2370288 ],
       [-1.0580108 , -1.5464424 , -0.65631646, -0.24539559, -1.3193743 ,
        -1.6654009 , -0.11233174, -0.14603867, -2.4399238 , -0.7109383 ],
       [-1.204778  , -0.8935618 , -1.5554076 ,  0.3468571 , -1.6494452 ,
        -1.1042782 , -0.9408648 ,  0.2623771 , -2.1418202 , -1.1196381 ]],
      dtype=float32))
('guide_log_scale_weight', array([[-5.663612 , -5.5355473, -5.9001265, -5.817989 , -5.1154127,
        -5.7710066, -6.050297 , -4.639577 , -6.0335135, -5.656336 ],
       [-5.5331335, -6.117948 , -6.292251 , -6.3286963, 

In [44]:
X = np.random.rand(200, indim)
w = 3 * np.ones((indim, outdim))
b = 1*np.ones((200, outdim))
y = np.matmul(X, w)
#     print(y.shape)
y = y + b

x_data, y_data = torch.tensor(X).type(torch.Tensor).unsqueeze(1), torch.tensor(y).type(torch.Tensor).unsqueeze(1)
loss = nn.MSELoss()
y_preds = torch.zeros(200,1,outdim)
for i in range(20):
    # guide does not require the data
    sampled_reg_model = guide(None)
    # run the regression model and add prediction to total
    y_preds = y_preds + sampled_reg_model(x_data)
    print(y_preds.size())
# take the average of the predictions
y_preds = y_preds / 20
print ("Loss: ", loss(y_preds, y_data).item())

torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
torch.Size([200, 1, 5])
Loss:  0.00330173852853477
